In [1]:
include("JQlaw.jl")
using .JQlaw
using Evolutionary
# using Plotly

## Simple Orbit Raise Example

In [2]:
orbit_initial = KeplarianOrbit(
                450000. + R_e,
                0.001,
                deg2rad(0.001),
                deg2rad(0.001),
                deg2rad(0.001),
                deg2rad(0)
                )

orbit_target = KeplarianOrbit(
                600000. + R_e,
                0.001,
                deg2rad(0.001),
                deg2rad(0.001),
                deg2rad(0.001),
                deg2rad(0)
                )

Q_Params = QParams(
                1,
                1,
                R_e+300000., 
                1,
                0,
                0,
                0,
                0,
                3,
                4,
                2,
                10^-10,
                0.5,
                0,
                10, 
                600.,
                10^4,
                10^3)

Sat_Params = sat_params(100, 0.8, 0.5/100., 3100)

time_initial = 59715.0 # MJD

Q_hist, orbit_hist, time_hist, mass_hist = Qlaw(orbit_initial, orbit_target, Q_Params, Sat_Params, time_initial)

print("Spent Prop: ", mass_hist[1] - mass_hist[end], "[kg]\n")
print("Transfer Time: ", time_hist[end] - time_hist[1], "[days]\n")

# change effectivity ratio to be more propellant safe...

Q_Params = QParams(
                1,
                1,
                R_e+300000., 
                1,
                0,
                0,
                0,
                0,
                3,
                4,
                2,
                10^-10,
                0.6,
                0,
                10, 
                600.,
                10^4,
                10^3)

Q_hist, orbit_hist, time_hist, mass_hist = Qlaw(orbit_inital, orbit_target, Q_Params, Sat_Params, time_initial)

print("Spent Prop: ", mass_hist[1] - mass_hist[end], "[kg]\n")
print("Transfer Time: ", time_hist[end] - time_hist[1], "[days]\n")

Spent Prop: 0.39397022723132125[kg]
Transfer Time: 0.6666666667442769[days]
Spent Prop: 0.2956234215741347[kg]
Transfer Time: 69.44444445252884[days]


In [3]:
initial_params = [3, 4, 2, 0.5, 0., 10., 500.]

7-element Vector{Float64}:
   3.0
   4.0
   2.0
   0.5
   0.0
  10.0
 500.0

In [4]:
lower = [0, 0, 0, 0, 0, 10, 500]
upper = [5, 5, 5, 1, 1, 50, 3000]

7-element Vector{Int64}:
    5
    5
    5
    1
    1
   50
 3000

In [5]:
BoxConstraints(lower, upper)

Box Constraints:
    x[1]≥0, x[1]≤5, x[2]≥0, x[2]≤5, x[3]≥0, x[3]≤5, x[4]≥0, x[4]≤1, x[5]≥0, x[5]≤1, x[6]≥10, x[6]≤50, x[7]≥500, x[7]≤3000

In [6]:
objective = make_objective(orbit_initial, orbit_target, Q_Params, Sat_Params, time_initial)
res = Evolutionary.optimize(objective,
    BoxConstraints(lower, upper),
    initial_params,
    GA(selection=uniformranking(2),
        populationSize=3,
        mutationRate=0.3,
        crossoverRate=0.4,
        mutation=Evolutionary.gaussian(0.5), crossover=SPX),
        Evolutionary.Options(
            iterations=2,
            show_trace=true,
            time_limit=10.0))

Iter     Function value
     0   0.36119577687757953
 * time: 0.005755901336669922
     1   0.36119577687757953
 * time: 0.07395792007446289
     2   0.36119577687757953
 * time: 0.11876177787780762



 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Minimizer:  [3.0, 4.0, 2.0,  ...]
    Minimum:    0.36119577687757953
    Iterations: 2

 * Found with
    Algorithm: GA[P=3,x=0.4,μ=0.3,ɛ=0]

 * Convergence measures
    |f(x) - f(x')| = 0.0 ≤ 1.0e-12

 * Work counters
    Seconds run:   0.119 (vs limit 10.0)
    Iterations:    2
    f(x) calls:    9


In [7]:
Evolutionary.minimizer(res)

7-element Vector{Float64}:
   3.0
   4.0
   2.0
   0.5
   0.0
  10.0
 500.0

In [8]:
convert(Int64, trunc(12.2))

12

In [9]:
trunc(12.2)

12.0

In [10]:
Evolutionary.trace(res)

LoadError: No trace in optimization results. To get a trace, run optimize() with store_trace = true.